# Assignment 1: Machine learning mini-project

1. How is your prediction task defined? Classification or regression, and what is the meaning of the output variable?
The prediction task is a classification where we read the twitter messages and analyse the sentiments; That is, classify the tweets as positive, negative or neutral.
The output variable Y is the labelling (postive/negative/neutral).

2. Where did you get the data? (Not necessary if you're working on the Twitter task.)
Twitter task

3. How do you represent your data as features?
Initially we used a bag of words representation for our features. In order to add clustering, we made it into a dictionary of clustername and the count which tells how often we encountered a word from that particular cluster.

4. Did you process the features in any way?
Yes, we used the feature_extraction(DictVectorizer) and feature_selection(SelectKBest) for processing the features.  

5. Did you bring in any additional sources of data?
Yes, we used the word clusters that were computed by Olutobi Owoputi and colleagues at Carnegie Mellon University, given in the assignment page.

6. How did you select which learning algorithms to use?
We used the different algorithms for classification and this one gave us the best results.

7. Did you try to tune the hyperparameters of the learning algorithm, and in that case how?
Yes, we used the Random Search procedure to tune the hyperparameters C and penalty of LogisticReqression.

8. How do you evaluate the quality of your system?
We used labeled test data to evaluate the trained classifier and it gives an accuracy score of 0.62.

9. How well does your system compare to a stupid baseline?
DummyClassifier gives an accuracy score of 0.377233620119 whereas our trained classifer gives 0.62.

10. Can you say anything about the errors that the system makes? For a classification task, you may consider a confusion matrix.
The confusion matrix for the labels ["positive", "negative", "neutral"] is:
[[200  84 242]
 [104  41 163]
 [247 104 326]]

Our classifier was mixing up Positive and Neutral (242 and 247) most often. But Positive and Negative (84 & 104) was rarely mixed up. Negative was very rarely guessed correctly. But then, Neutral and Positive were good.

11. Is it possible to say something about which features the model considers important? (Whether this is possible depends on the type of classifier you are using.)
The classifier chooses the ones with the highest count values for the clusters.


In [15]:
import re
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon
from collections import defaultdict
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix


tokenize_re = re.compile(r'''
                         \d+[:\.]\d+
                         |(https?://)?(\w+\.)(\w{2,})+([\w/]+)
                         |[@\#]?\w+(?:[-']\w+)*
                         |[^a-zA-Z0-9 ]+''',
                         re.VERBOSE)

def tokenize(text):
    return [ m.group() for m in tokenize_re.finditer(text) ]

def print_token(filename):
    with open(filename) as f:
        for l in f:
            print(tokenize(l))
            break
            
            
clusters = defaultdict(str)


def create_clusters(filename):
    with open(filename, encoding="utf8") as f:
            for l in f:
                cols = l.strip('\n').split('\t')
                clustername = cols[0]
                token = cols[1]
                # check if the token is already in a cluster; if not -> add it
                if token not in clusters:
                    clusters[token] = clustername
 



# Funct        
def read_word_clusters(filename):
    tokens = []
    X = []
    Y = []
    with open(filename) as f:
        for l in f:
            l = l.strip('\n')
            tokens = tokenize(l)
            tweet_clusters = defaultdict(int)
            
            
            # Read from tokens[2] and compare every word with clusters 
            # If the word is in clusters, get the clustername else ignore it
            # If the clustername & count is present in tweet_clusters, update the count, else
            # insert the entry into tweet_clusters
            for t in tokens[2:]:
                if t in clusters:
                    c = clusters[t]
                    if c in tweet_clusters:
                        count = tweet_clusters[c]
                        tweet_clusters.update({c:count+1})
                    else:
                        tweet_clusters[c] = 1
            X.append(tweet_clusters)
            Y.append(tokens[0])
    return X, Y
        
# Initially we represented the features as a bag of words and used CountVectorizer    
def read_feature_descriptions(filename):
    tokens = []
    X = []
    Y = []
    with open(filename) as f:
        for l in f:
            l = l.strip('\n')
            tokens = tokenize(l)
            X.append(tokens[2:])
            Y.append(tokens[0])

    #Xe = vec.fit_transform(X)
    return X, Y
#vec = CountVectorizer(preprocessor = lambda x: x, tokenizer = lambda x: x)    


# Create the word clusters
create_clusters('/Users/nickeylawrence/Desktop/Chalmers/Academics/ML/Asg2/50mpaths2.txt')

# Read the training and test data (X,Y)
Xtrain, Ytrain = read_word_clusters('/Users/nickeylawrence/Desktop/Chalmers/Academics/ML/Asg2/train.tsv')
Xtest, Ytest = read_word_clusters('/Users/nickeylawrence/Desktop/Chalmers/Academics/ML/Asg2/dev.tsv')

#XCluster, YCluster = read_word_clusters('/Users/nickeylawrence/Desktop/Chalmers/Academics/ML/Asg2/train.tsv')
#Xtrain, Ytrain = read_feature_descriptions('/Users/nickeylawrence/Desktop/Chalmers/Academics/ML/Asg2/train.tsv')
#Xtest, Ytest = read_feature_descriptions('/Users/nickeylawrence/Desktop/Chalmers/Academics/ML/Asg2/dev.tsv')


# Preprocessing for tuning hyperparameters using RandomSearch
preprocessing_pipeline = make_pipeline(
    DictVectorizer(),
    StandardScaler(with_mean=False),
    SelectKBest(k=100),
)
preprocessing_pipeline.fit(Xtrain, Ytrain)
X_vec = preprocessing_pipeline.transform(Xtrain)
clf = LogisticRegression()
C_distr = expon(scale=3)
param_grid_random = {'C': C_distr, 'penalty': ['l1', 'l2']}
randomsearch = RandomizedSearchCV(clf, param_grid_random, n_iter=10)
randomsearch.fit(X_vec, Ytrain);
best_C = randomsearch.best_params_['C']
best_penalty = randomsearch.best_params_['penalty']


# Training and Evaluating the Classifier
pipeline = make_pipeline(DictVectorizer(), SelectKBest(k=500), 
                         LogisticRegression(C=best_C, penalty=best_penalty))

pipeline.fit(Xtrain, Ytrain)
Yguess = pipeline.predict(Xtest)
cross_validate(pipeline, Xtrain, Ytrain)
print(accuracy_score(Ytest, Yguess))

#DummyClassifer
dummy_pipeline = make_pipeline(
    DictVectorizer(),
    DummyClassifier()
)
dummy_pipeline.fit(Xtrain, Ytrain)
Yguess = dummy_pipeline.predict(Xtest)
cross_validate(dummy_pipeline, Xtrain, Ytrain)
print(accuracy_score(Ytest, Yguess))


#Confusion Matrix
print(confusion_matrix(Ytest, Yguess, labels = ["positive", "negative", "neutral"]))


0.618795499669
0.374586366645
[[207  74 245]
 [117  44 147]
 [266  96 315]]
